<a href="https://colab.research.google.com/github/joromor/Dataset-Pinus/blob/main/Notebook_detect_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exportar ortomosaico a ser trabalhado e modelo melhor peso para o Google Drive

Conectar ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# buscando o ortomosaico no Google Drive
path_img = '/caminho da ortofoto no Google Drive'

In [ ]:
# instalando as bibliotecas
!pip install rasterio
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 42.7 MB/s eta 0:00:00


In [ ]:
# instalando demais bibliotecas e módulos
import rasterio
from matplotlib import pyplot as plt
import os
from rasterio.windows import Window
from skimage.io import imsave
from skimage import io
import glob
import numpy as np
import geopandas as gpd
from rasterio.plot import show
import pandas as pd
from shapely.geometry import Polygon
import os
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import json

Verificando os dados do ortomosaico (p.ex: 4 bandas x 5697 pixels e largura e 6194 pixels dealtura)

In [ ]:
src_img = rasterio.open(path_img)

In [ ]:
img = src_img.read()

In [ ]:
img.shape

(4, 5697, 6194)

p.ex. ortomosaico com 4 bandas e tamanho de 5697 pixels de largura e 6194 pixels de altura

É necessário inverter a ordem: largura x altura x n. bandas

In [ ]:
img = img.transpose([1,2,0])

In [ ]:
img.shape

(5697, 6194, 4)

In [ ]:
# criar uma pasta Predict para receber as imagens recortadas tiff
if not os.path.isdir('/content/Predict'):
    os.mkdir('/content/Predict')

Dividir o ortomosaico em partes de 640 x 640 pixels e salvando-as na pasta predict

In [ ]:
# Dividir o ortomosaico em partes de 640 x 640 pixels e salvando-as na pasta Predict criada
qtd = 0
out_meta = src_img.meta.copy()
for n in range((src_img.meta['width']//640)):
  for m in range((src_img.meta['height']//640)):
    x = (n*640)
    y = (m*640)
    window = Window(x,y,640,640)
    win_transform = src_img.window_transform(window)
    arr_win = src_img.read(window=window)
    arr_win = arr_win[0:3,:,:]
    if (arr_win.max() != 0) and (arr_win.shape[1] == 640) and (arr_win.shape[2] == 640):
      qtd = qtd + 1
      path_exp = '/content/Predict/img_' + str(qtd) + '.tif'
      out_meta.update({"driver": "GTiff","height": 640,"width": 640, "transform":win_transform})
      with rasterio.open(path_exp, 'w', **out_meta) as dst:
          for i, layer in enumerate(arr_win, start=1):
              dst.write_band(i, layer.reshape(-1, layer.shape[-1]))
      del arr_win

Conferir a quantidade de inagens de 640 x 640 geradas (ex: 64)

In [ ]:
# Conferir a quantidade de imagens de 640 x 640 geradas (ex;64)
print(qtd)

64


In [ ]:
# Criando uma pasta Predict_jpg para receber as imagens jpg convertidas a ártir das imagens tif (procedimento necessário pois o YOLO não trabalha com tif)
# converter as imagens tif para jpg
if not os.path.isdir('/content/Predict_jpg'):
    os.mkdir('/content/Predict_jpg')

path_data_pred = '/content/Predict_jpg'
imgs_to_pred = os.listdir('/content/Predict')
for images in imgs_to_pred:
  src = rasterio.open('/content/Predict/' + images)
  raster = src.read()
  raster = raster.transpose([1,2,0])
  raster = raster[:,:,0:3]
  imsave(os.path.join(path_data_pred,images.split('.')[0] + '.jpg'), raster)

/usr/local/lib/python3.11/dist-packages/skimage/_shared/utils.py:328: UserWarning: /content/Predict_jpg/img_20.jpg is a low contrast image
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/skimage/_shared/utils.py:328: UserWarning: /content/Predict_jpg/img_12.jpg is a low contrast image
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/skimage/_shared/utils.py:328: UserWarning: /content/Predict_jpg/img_5.jpg is a low contrast image
  return func(*args, **kwargs)


In [ ]:
# Instalar a biblioteca ultralytics
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# Importando o modelo YOLO
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Comando para detecção: tarefa: detect, modo: predict, modelo: melhor modelo, confiança (50%), local onde as imagens jpg estão armazenadas, salvae imagem e salvar rótulo.
!yolo task=detect mode=predict model="/content/drive/MyDrive/best773_200.pt" conf=0.50 source= "/content/Predict_jpg" save=True save_txt=True

Ultralytics 8.3.86 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 112 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs

image 1/64 /content/Predict_jpg/img_1.jpg: 640x640 1 Pinus, 4056.4ms
image 2/64 /content/Predict_jpg/img_10.jpg: 640x640 (no detections), 5088.2ms
image 3/64 /content/Predict_jpg/img_11.jpg: 640x640 2 Pinuss, 3756.3ms
image 4/64 /content/Predict_jpg/img_12.jpg: 640x640 (no detections), 3731.4ms
image 5/64 /content/Predict_jpg/img_13.jpg: 640x640 1 Pinus, 5262.3ms
image 6/64 /content/Predict_jpg/img_14.jpg: 640x640 1 Pinus, 3791.8ms
image 7/64 /content/Predict_jpg/img_15.jpg: 640x640 (no detections), 3753.0ms
image 8/64 /content/Predict_jpg/img_16.jpg: 640x640 1 Pinus, 5193.3ms
image 9/64 /content/Predict_jpg/img_17.jpg: 640x640 1 Pinus, 3821.8ms
image 10/64 /content/Predict_jpg/img_18.jpg: 640x640 3 Pinuss, 3764.8ms
image 11/64 /content/Predict_jpg/img_19.jpg: 640x640 (no detections), 4644.3ms
image 12/64 /content/Predict_jp

In [ ]:
# Listar as imagens jpg e o a pasta onde os resultados estão e associar os rótulos às imagens tif e reconverter para as coordenadas do ortomosaico (x 640)
ls_x = []
ls_y = []
imgs_to_pred = [f for f in os.listdir('/content/runs/detect/predict/labels') if f.endswith('.txt')]
for images in imgs_to_pred:
  filename = images.split('.')[0]
  src = rasterio.open('/content/Predict/' + filename + '.tif')
  path = f'/content/runs/detect/predict/labels/'+filename+'.txt'
  cols = ['class', 'x-center', 'y-center', 'x_extend', 'y_extend']
  df = pd.read_csv(path, sep=" ", header=None)
  df.columns = cols
  df['x-center'] = np.round(df['x-center'] * 640)
  df['y-center'] = np.round(df['y-center'] * 640)
  for i,row in df.iterrows():
    xs, ys = rasterio.transform.xy(src.transform, row['y-center'], row['x-center'])
    ls_x.append(xs)
    ls_y.append(ys)

In [ ]:
df_xy = pd.DataFrame([])
df_xy['x'] = ls_x
df_xy['y'] = ls_y

In [ ]:
gdf = gpd.GeoDataFrame(df_xy, geometry=gpd.points_from_xy(df_xy['x'], df_xy['y']))

In [ ]:
# Conferindo o Sistema de Coordenadas (Ex: SIRGAS 2000)
print(src_img.crs)

EPSG:31982


In [ ]:
gdf = gdf.set_crs(src_img.crs.to_dict())

In [ ]:
# Visualizando as detecções no ortomosaico - centroide das caixas delimitadoras
fig, ax = plt.subplots(figsize=(40, 40))
show((src_img), ax=ax)
gdf.plot(ax=ax,  marker='o', color='red', markersize=60)

In [ ]:
# Salvendo os centroides em formato JSON
gdf.to_file('/content/pinus_detect.json', driver="GeoJSON")

In [ ]:
# Exportar as detecções para uma pasta no Google Drive específica
!cp -R /content/runs/detect/predict # '/content/drive/MyDrive criar uma pasta'

In [ ]:
# A partir das coordenadas dos centróides criar um shapefile com as quatro coordenadas da caixa delimitadora
# Lista para armazenar os polígonos das caixas delimitadoras
boxes_list = []

# Listar arquivos de labels
imgs_to_pred = [f for f in os.listdir('/content/runs/detect/predict/labels') if f.endswith('.txt')]

for images in imgs_to_pred:
    filename = images.split('.')[0]
    src = rasterio.open(f'/content/Predict/{filename}.tif')
    path = f'/content/runs/detect/predict/labels/{filename}.txt'

    # Ler arquivo de detecções
    cols = ['class', 'x-center', 'y-center', 'x_extend', 'y_extend']
    df = pd.read_csv(path, sep=" ", header=None)
    df.columns = cols

    # Converter para pixels da imagem (assumindo 640x640)
    df['x-center'] = np.round(df['x-center'] * 640)
    df['y-center'] = np.round(df['y-center'] * 640)
    df['x_extend'] = np.round(df['x_extend'] * 640)
    df['y_extend'] = np.round(df['y_extend'] * 640)

    for _, row in df.iterrows():
        # Calcular coordenadas da bounding box (em pixels)
        x_min = row['x-center'] - (row['x_extend'] / 2)
        x_max = row['x-center'] + (row['x_extend'] / 2)
        y_min = row['y-center'] - (row['y_extend'] / 2)
        y_max = row['y-center'] + (row['y_extend'] / 2)

        # Converter para coordenadas geográficas
        x_min_geo, y_min_geo = rasterio.transform.xy(src.transform, y_max, x_min)
        x_max_geo, y_max_geo = rasterio.transform.xy(src.transform, y_min, x_max)
        x_min_geo, y_max_geo = rasterio.transform.xy(src.transform, y_min, x_min)
        x_max_geo, y_min_geo = rasterio.transform.xy(src.transform, y_max, x_max)

        # Criar polígono da bounding box
        polygon = Polygon([
            (x_min_geo, y_min_geo),
            (x_min_geo, y_max_geo),
            (x_max_geo, y_max_geo),
            (x_max_geo, y_min_geo),
            (x_min_geo, y_min_geo)  # Fechar o polígono
        ])

        # Armazenar os dados no GeoDataFrame
        boxes_list.append({
            "filename": filename,
            "class": row["class"],
            "geometry": polygon
        })

# Criar GeoDataFrame
gdf = gpd.GeoDataFrame(boxes_list, crs=src.crs)  # Define o CRS a partir do raster original

# Salvar como Shapefile
output_shp = "/content/detected_boxes.shp"
gdf.to_file(output_shp)

print(f"Shapefile salvo em: {output_shp}")


Shapefile salvo em: /content/detected_boxes.shp


Alternativamente pode-se criar os centroides em shapefile com o código abaixo

In [ ]:
# Lista para armazenar os pontos dos centróides
centroid_list = []

# Diretórios dos arquivos
labels_dir = "/content/runs/detect/predict/labels"
raster_dir = "/content/Predict"
output_shp = "/content/centroides.shp"  # Novo shapefile só com centróides

# Listar arquivos de labels
imgs_to_pred = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

for images in imgs_to_pred:
    filename = images.split('.')[0]
    raster_path = os.path.join(raster_dir, f"{filename}.tif")
    label_path = os.path.join(labels_dir, f"{filename}.txt")

    if not os.path.exists(raster_path):
        print(f"⚠️ Raster não encontrado: {raster_path}")
        continue

    src = rasterio.open(raster_path)

    # Ler arquivo de detecções
    cols = ['class', 'x-center', 'y-center', 'x_extend', 'y_extend']
    df = pd.read_csv(label_path, sep=" ", header=None)
    df.columns = cols

    # Converter coordenadas normalizadas para pixels (assumindo 640x640)
    df['x-center'] = np.round(df['x-center'] * 640)
    df['y-center'] = np.round(df['y-center'] * 640)

    for _, row in df.iterrows():
        # Converter coordenadas do centro para geográficas
        x_geo, y_geo = rasterio.transform.xy(src.transform, row['y-center'], row['x-center'])

        # Criar ponto do centróide
        point = Point(x_geo, y_geo)

        # Armazenar os dados no GeoDataFrame
        centroid_list.append({
            "filename": filename,
            "class": row["class"],
            "geometry": point
        })

# Criar GeoDataFrame com centróides
gdf_centroids = gpd.GeoDataFrame(centroid_list, crs=src.crs)  # CRS da imagem original

# Se CRS estiver indefinido, definir manualmente como SIRGAS 2000 / UTM 22S (EPSG:31982)
if gdf_centroids.crs is None:
    gdf_centroids.set_crs(epsg=31982, inplace=True)

# Salvar como Shapefile
gdf_centroids.to_file(output_shp, driver="ESRI Shapefile")

print(f"✅ Shapefile de centróides salvo com sucesso em: {output_shp} (EPSG:31982)")


✅ Shapefile de centróides salvo com sucesso em: /content/centroides.shp (EPSG:31982)
